# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import pickle

# Load Data and Preprocessing

In [ ]:
from loadAndPreprocess import load_and_preprocess

'''
well_info: Well, X, Y, Total Resources
sensor_data: Depth, Porosity, Hydrate Saturation, Estimated Resources
'''
well_info, sensor_data_list = load_and_preprocess()

# Remove the data point with NaN value
well_info = well_info.dropna()
for sensor_data in sensor_data_list:
    sensor_data.dropna(inplace=True)

# Kriging 3D Interpolation

### Define Functions

In [ ]:
def plot_3D_mesh(t, grid_x, grid_y, grid_z, title: str, color_scale='Viridis'):
    GZ, GY, GX = np.meshgrid(grid_z, grid_y, grid_x, indexing='ij')

    fig = go.Figure(
        data=[go.Volume(
            x=GX.flatten(),
            y=GY.flatten(),
            z=GZ.flatten(),
            value=t.flatten(),
            isomax=t.max(),
            isomin=t.min(),
            opacity=0.5,  
            surface_count=25,  # Adjust the number of isosurfaces
            colorscale=color_scale 
        )])

    # Update the layout of the plot
    fig.update_layout(
        height=600,
        width=700,
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis=dict(
                title='Depth',
                autorange='reversed'  # Automatically reverse the z-axis
            )
        )
    )

    # Show the plot
    fig.show()

### Define the Space to Interpolate

In [ ]:
step_sizes = [60.0, 60.0, 6.0]

X = well_info['X'].values
Y = well_info['Y'].values
Z = [sensor_data['Depth'] for sensor_data in sensor_data_list]
Z = np.concatenate(Z)

# Define grid points for interpolation
grid_x = np.arange(min(X)-200, max(X)+200, step_sizes[0])
grid_y = np.arange(min(Y)-200, max(Y)+200, step_sizes[1])
grid_z = np.arange(min(Z)-10, max(Z)+10, step_sizes[2])

grid = (grid_x, grid_y, grid_z)

### Kriging Interpolation of Porosity

In [ ]:
# Load the model
with open('models/ok3d_poro.pkl', 'rb') as f:
    ok3d_poro = pickle.load(f)

In [ ]:
t_poro, ss3d_poro = ok3d_poro.execute('grid', *grid)

In [ ]:
plot_3D_mesh(t_poro, *grid, "Kriging Interpolation of Porosity", "Plasma")

In [ ]:
plot_3D_mesh(ss3d_poro, *grid, "Kriging Varience of Porosity")

### Kriging Interpolation of Hydrate Saturation

In [ ]:
# Load the model
with open('models/ok3d_hydr.pkl', 'rb') as f:
    ok3d_hydr = pickle.load(f)

In [ ]:
t_hydr, ss3d_hydr = ok3d_hydr.execute('grid', *grid)

In [ ]:
plot_3D_mesh(t_hydr, *grid, "Kriging Interpolation of Hydrate Saturation", "Plasma")

In [ ]:
plot_3D_mesh(ss3d_hydr, *grid, "Kriging Varience of Hydrate Saturation")